In [ ]:
def get_data_and_scoring_function(dataset_Id, additional_data):
  if dataset_Id == 0:
    X = additional_data.iloc[:, :-1]
    y = additional_data.loc[:, "class"]
  else:
    X, y = sklearn.datasets.fetch_openml(data_id= dataset_Id, as_frame=True, return_X_y=True)

  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
      X, y, stratify=y, test_size = 0.3
  )

  def scoring_function_auc(estimator):
      predictions = estimator.predict_proba(X_test)[:, 1]
      return sklearn.metrics.roc_auc_score(y_test, predictions)

  def scoring_function_acc(estimator):
    y_pred= estimator.predict(X_test)
    acc_score= accuracy_score(y_test, y_pred)
    return acc_score


    #def train_scoring_function(estimator):
        #predictions = estimator.predict_proba(X_train)[:, 1]
        #return sklearn.metrics.roc_auc_score(y_train, predictions)



  return X_train, y_train, X_test, y_test, scoring_function_acc, scoring_function_auc

In [ ]:
def checkAutoSklearnPreprocessing(res_acc_list, res_auc_list, result_log_file , X_train, y_train, X_test, y_test, scoring_function_acc, scoring_function_auc, time_limit):
    ## result_list_acc, result_list_roc - empty lists that will contain the results (accuracy and auc scores), result_log_file - the file in which there will be information about preprocessing,
    ## scoring_function_acc/auc - functions created earlier, that calculate accuracy and auc scores, time_limit - time limit of training
    settings = {
    "time_left_for_this_task": time_limit,  # seconds
    "seed": 42,
    "metric": roc_auc,
    "n_jobs": 4,
    }

    resampling_strategy = "holdout"

    autoSklearnClassifierInstance = AutoSklearnClassifier(
        **settings,
        resampling_strategy=resampling_strategy
    )

    autoSklearnClassifierInstance.fit(X_train, y_train)

    models = autoSklearnClassifierInstance.show_models()
    result_file = open(result_log_file,"w")
    print(models)
    for model_id in models:
      print(model_id)
      runhistory = autoSklearnClassifierInstance.automl_.runhistory_
      full_config = runhistory.ids_config[model_id]
      full_config
      result_file.write(str(full_config))
      print(full_config)
      result_file.write(f"Accuracy:{scoring_function_acc(autoSklearnClassifierInstance)}, AUC:{scoring_function_auc(autoSklearnClassifierInstance)} ")
    result_file.close()

    # zapisujemy wyniki roc i auc do list

    res_acc_list.append(scoring_function_acc(autoSklearnClassifierInstance))
    res_auc_list.append(scoring_function_auc(autoSklearnClassifierInstance))





